In [1]:
import numpy as np
import glob
import os
import uproot
import json

In [14]:
# Define auxilary functions 

def get_limits(fn): # Returns quantile vs limits
    f = uproot.open(fn)
    limit = f["limit"]['limit'].array(library="np")
    quant = f["limit"]['quantileExpected'].array(library="np")
    if limit.shape[0] == 1:
        return -1
    else:
        return np.stack([quant,limit]) 
    
def get_SUEP_file(Mass=125, mdark=2, t=1, decay='generic', HT='1000', path="../"): # Returns filename
    if t < 10:
        tem = "{0:.2f}".format(t)
    else:
        tem = "{0:.1f}".format(t)
    tem = str(tem).replace(".","p")
    fname = os.path.join(
        "{}higgsCombineGluGluToSUEP_HT{}_T{}_mS{:.3f}_mPhi{:.3f}_T{:.3f}_mode{}_TuneCP5_13TeV-pythia8.AsymptoticLimits.mH125.root".format(path, HT, tem, Mass, mdark, t, decay)
    )
    if os.path.isfile(fname):
        return fname
    else:
        pass
     
def xs_scale(proc, era):
    xsec = 1.0
    with open(f"../config/xsections_{era}.json") as file:
            MC_xsecs = json.load(file)
    xsec  = MC_xsecs[proc]["xsec"]
    xsec *= MC_xsecs[proc]["kr"]
    xsec *= MC_xsecs[proc]["br"]
    #xsec *= 1000.0
    assert xsec > 0, "{} has a null cross section!".format(proc)
    return xsec

# Get all SUEP masses for a given mdark and T
def get_params_from_dir(path="../", era='2018', HT='1000'):
    """
    Grabs the parameters [T, mS, mPhi, mode, xsec] from each file from combine
    from a specific directory. The 'era' parameter is just used for the
    cross sections, which for SUEP don't change between years. HT parameter
    specifies the HT of the files.
    """

    # Get files
    combine_files = glob.glob("{}higgsCombineGluGluToSUEP_HT{}*.root".format(path, HT))

    # Filter out the parameters
    all_params = []
    for file in combine_files:
        file_trim = file.split("HT{}_".format(HT))[1]
        proc = 'GluGluToSUEP_HT{}_'.format(HT) + file_trim.split('.Asymptotic')[0]
        xsec = xs_scale(proc, era)
        t = float(file_trim.split("_")[0].replace("p",".")[1:])
        Mass = float(file_trim.split("_")[1][2:])
        mphi = float(file_trim.split("_")[2][4:])
        mode = file_trim.split('_')[4][4:]
        all_params.append([t,Mass,mphi,mode, xsec])

    return all_params


def create_limit_table(params, path, HT='1000'):
    """
    Expects params to be a list of lists of the shape: [t,Mass,mphi,mode, xsec].
    Returns table of limits.
    """

    rows = [['T', 'mS', 'mPhi', 'Mode', 'Xsec', 'Obs', 'Exp', '+2sigma', '+1sigma', '-1sigma', '-2sigma']]
    for param in params:
        lims = get_limits(get_SUEP_file(path=path, HT=HT, Mass=param[1], mdark=param[2], t=param[0], decay=param[3]))
        if lims.shape != (2,6):
            print(param, "has bad limits")
            continue

        lims = lims * param[-1]        

        _exp =lims[1][2]
        _s1p =lims[1][1] 
        _s1m =lims[1][3] 
        _s2p =lims[1][0] 
        _s2m =lims[1][4] 
        _obs =lims[1][5]   

        newline = [*param, _obs, _exp, _s2p, _s1p, _s1m, _s2m]
        rows.append(newline)
    
    return rows
    

In [15]:
# Take temperature, mphi/mdark, decay mode (in GeV) 
ofile = '../ClosureSyst/hep_data.txt'
itag = '../ClosureSyst/'
HT = '1000'

In [16]:
params = get_params_from_dir(itag, HT=HT)
table = create_limit_table(params, path=itag, HT=HT)
print(table[:10])

[['T', 'mS', 'mPhi', 'Mode', 'Xsec', 'Obs', 'Exp', '+2sigma', '+1sigma', '-1sigma', '-2sigma'], [3.0, 700.0, 3.0, 'generic', 0.024897660529286627, 3.879796834835871e-05, 3.837072316677779e-05, 1.1241422802766932e-05, 1.993127682561416e-05, 7.446629876079715e-05, 0.00011651486626107549], [3.0, 900.0, 3.0, 'generic', 0.017255090398804143, 2.6528642639867242e-05, 2.6460824357297005e-05, 7.752194635926857e-06, 1.3744802593553232e-05, 5.11417374339598e-05, 8.034896636768543e-05], [3.0, 125.0, 3.0, 'generic', 0.11734286459407163, 6.673498758558057, 8.903389851075184, 4.834262458200979, 6.391975288285636, 12.665254981657974, 17.421236341835687], [3.0, 400.0, 3.0, 'generic', 0.046354350400438414, 0.001020698753187191, 0.0008459442608478446, 0.0004560168281132912, 0.0006113785395934648, 0.0011629094120707915, 0.0015400562864700776], [3.0, 200.0, 3.0, 'generic', 0.08151254385944313, 0.7057128600224649, 0.5693141735182982, 0.315791455623431, 0.41680441353466713, 0.7883026425037206, 1.061780066086

In [17]:
with open(ofile, "w") as file:
    for row in table:
        # Convert the elements of the inner list to strings
        row_str = [str(element) for element in row]
        # Join the elements with a delimiter (e.g., comma) and write to the file
        file.write(",".join(row_str) + "\n")